### Async
- Running functions parallely so that responses can come irrespective of completion of non related functions
- Speeds up the code

`Multiprocessing`
- The Operating System assigns the task parallely and they execute together

In [1]:
from multiprocessing import Process     #Importing a Process Object

In [2]:
def showSquare(n = 2):
    print(n**2)
    for i in range(1000): pass

In [3]:
procs = []

In [4]:
for i in range(5):
    procs.append(Process(target=showSquare, args=(i+1,)))   #To give arguments, we use args and send a tuple

In [5]:
for proc in procs:
    proc.start()

print("Hello")

Hello


In this we see that Hello can br printed anytime between the 5 processes, hence we dont have much controller over it, only the **Operating System** does

For waiting for the proceses to finish, we use `join`

In [6]:
for proc in procs:
    proc.join()

print("Hello World")

Hello World


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'showSquare' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'showSquare' on <module '__main__' (built-in)>
Traceback (most rece

The clear difference between the times show how this waits for all the process to finish

---

`Threading`
- A thread is a chunk of code that can be executed
- In multi threading, we use multiple threads for the program and hence send the processes to different threads
- **IMP:** The data is shared among threads (context sharing)

> By default python runs cpython, which utilises single thread
> 
> Other variations of the interpreter like `jython` can implement multi-threading

In [7]:
from threading import Thread
# For creating a thread

In [8]:
def square(n):
    print("Square:", n**2)

def cube(n):
    print("Cube:", n**3)

In [9]:
t1 = Thread(target=square, args=(2,))

In [10]:
t2 = Thread(target=cube, args=(3,))

In [11]:
t1.start()
t2.start()
print("Hello")

Square:Cube: 4
Hello
 27


As we can see, the thread utilises the same `STDOUT`, it can even interfere within the function while running

This affirms that all this is running asynchronously

In [12]:
t1.join()
t2.join()
print("Hello")

Hello


Join makes it wait synchronously, and the executes the lines afterwrds

To illustrate the context sharing in threads, we use a queue, a queue in python is inbuilt with mutex, that holds a thread until the work is complete or the function needs time to do some stuff not requiring the thread, and the thread attends the other function

In [13]:
# Example of resource sharing in threads
from queue import Queue

In [14]:
def producer(q):
    for i in range(5):
        q.put(i)
        print("Published", i)

def consumer(q):
    while True: #Runs indefinitely, keeps on consuming
        i = q.get()
        print("Consumed", i)

In [15]:
q = Queue()

producer_thread = Thread(target=producer, args=(q,))
consumer_thread = Thread(target=consumer, args=(q,))

In [16]:
consumer_thread.start()
#Starting the consumer thread, which will run indefinitely

In [17]:
producer_thread.start()
#Now starting publishing, allowing consumer to consume some values

Published 0
PublishedConsumed 0
Consumed 1
 1
Published 2
Published 3
Published 4
Consumed 2
Consumed 3
Consumed 4


As we can see, the consumer keeps on consuming, and producer keeps on publishing, but these share a data structure queue, and this is how contect sharing is done

In [18]:
producer_thread.join()

In [19]:
#consumer_thread.join()

The producer thread has finished publishing, hence it returns the join, but consumer thread is running indefinitely, hence it will just keep on running until stopped, you can run the below line to test it out

---

`Coroutines`
- When the user implements context switching themselves
- Using the `yield` keywords, a function can be sent the context of the current function
- The yield function can return as well as *consume* the data

In [20]:
def func():
    return 5

def func2():
    yield func()    #Passes the context

def main():
    pass

In [21]:
func2()

<generator object func2 at 0x103b033e0>

`yield` can also consume data

In [27]:
def fancy_name(prefix):
    try:
        while True: #Runs indefinitely
            name = (yield)
            print(prefix + ":" + name)

    except GeneratorExit:
        print("Done")

In [28]:
co = fancy_name("Cool")

In [29]:
#Initialisation
next(co)

In [30]:
#Sending data
co.send("Utkarsh")

Cool:Utkarsh


In [31]:
co.send("Gupta")

Cool:Gupta


In [32]:
co.close()

Done


`AsyncIO`
- Available after Python 3.7+